# Time dependent D3D simulation

In [ ]:
using Plots;
using FUSE
using Interact
FUSE.ProgressMeter.ijulia_behavior(:clear);

In [ ]:
use_local_cache = false
shot = 168830; @time ini, act = FUSE.case_parameters(:D3D, shot; fit_profiles=true, use_local_cache)
@checkin :fetch ini act

In [ ]:
# initialization takes experimental data and populates dd.pulse_schedule in a way that allows us to postdict the experiments
# If we ever wanted to play "what if" simulations in FUSE, we would change the dd.pulse_schedule

ini.time.simulation_start = ini.general.dd.equilibrium.time_slice[2].time
dd = IMAS.dd()
@time FUSE.init!(dd, ini, act);
@checkin :init dd act;
dd1 = FUSE.checkpoint[:init].dd;

In [ ]:
@manipulate for time0 in dd.equilibrium.time
    plot(dd.core_profiles.profiles_1d[time0]; thomson_scattering=true, charge_exchange=true)
end

In [ ]:
@checkout :init dd act;
experiment_LH = FUSE.LH_analysis(dd; do_plot=true);

In [ ]:
@checkout :init dd act;

act.ActorPedestal.model = :dynamic
act.ActorPedestal.tau_n = experiment_LH.tau_n
act.ActorPedestal.tau_t = experiment_LH.tau_t
act.ActorWPED.ped_to_core_fraction = experiment_LH.W_ped_to_core_fraction
act.ActorEPED.ped_factor = 1.0
act.ActorPedestal.T_ratio_pedestal = 1.0 # Ti/Te in the pedestal

if true
    # density and Zeff from experiment
    act.ActorPedestal.density_ratio_L_over_H = 1.0
    act.ActorPedestal.zeff_ratio_L_over_H = 1.0
else
    # density can go from L to H mode at a different time
    act.ActorPedestal.density_ratio_L_over_H = experiment_LH.ne_L_over_H
    act.ActorPedestal.zeff_ratio_L_over_H = experiment_LH.zeff_L_over_H
    dd.pulse_schedule.density_control.n_e_line.reference = experiment_LH.ne_H
    dd.pulse_schedule.density_control.zeff_pedestal.reference = experiment_LH.zeff_H
end

if false
    # LH-transition from LH scaling law
    act.ActorPedestal.mode_transitions = missing
else
    # LH-transition at user-defined times
    act.ActorPedestal.mode_transitions = experiment_LH.mode_transitions
    act.ActorPedestal.mode_transitions[5.2] = :L_mode
end

act.ActorEquilibrium.model = :FRESCO #:EGGO or FRESCO
act.ActorFRESCO.nR = 65
act.ActorFRESCO.nZ = 65

act.ActorNeutralFueling.τp_over_τe = 0.25

act.ActorFluxMatcher.evolve_plasma_sources = false
act.ActorFluxMatcher.algorithm = :simple
act.ActorFluxMatcher.max_iterations = -10 # negative to avoid print of warnings
act.ActorFluxMatcher.evolve_pedestal = false
act.ActorFluxMatcher.relax=0.5

act.ActorTGLF.tglfnn_model = "sat1_em_d3d"

# time
δt = 0.05 # 0.025
dd.global_time = ini.general.dd.equilibrium.time_slice[2].time # start_time should be early in the shot, when otherwise ohmic current will be wrong
final_time = ini.general.dd.equilibrium.time[end]
act.ActorDynamicPlasma.Nt = Int(ceil((final_time - dd.global_time) / δt))
act.ActorDynamicPlasma.Δt = final_time - dd.global_time

act.ActorDynamicPlasma.evolve_current = true
act.ActorDynamicPlasma.evolve_equilibrium = true
act.ActorDynamicPlasma.evolve_transport = true
act.ActorDynamicPlasma.evolve_hcd = true
act.ActorDynamicPlasma.evolve_pf_active = false
act.ActorDynamicPlasma.evolve_pedestal = true

# act.ActorCurrent.model = :replay
# act.ActorEquilibrium.model = :replay
# act.ActorCoreTransport.model = :replay
# act.ActorPedestal.model = :replay
# act.ActorHCD.ec_model = :replay
# act.ActorHCD.ic_model = :replay
# act.ActorHCD.lh_model = :replay
# act.ActorHCD.nb_model = :replay
# act.ActorHCD.pellet_model = :replay
# act.ActorHCD.neutral_model = :none

# for unit in dd.pulse_schedule.nbi.unit
#     unit.power.reference .*= .5
# end

# GC.gc()

@time actor = FUSE.ActorDynamicPlasma(dd, act; verbose=true);

In [ ]:
@manipulate for time0 in slider(dd.equilibrium.time, value=dd.global_time/2.0, label="time")
    try
        FUSE.plot_plasma_overview(dd, Float64(time0); dd1, aggregate_hcd=true)#, min_power=1E4)
        
#         IMAS.ylim(Dict{Int,Float64}(
#             -2 => 0.0,
#             -3 => 0.0, 3 => 2.0,
#             -4 => 0.0, 4 => 4.0,
#             -5 => 0.0, 5 => 4.0,
#             6 => 1.1E20,

#             -8 => -0.2, 8 => 2.0,
#             -9 => -0.25, 9 => .5,
#             -10 => -0.25, 10 => .5,
#             -11 => -1E20, 11 => 1.0E20,

#             -14 => 0.0, 14 => 0.101,
#             -15 => 0.0, 15 => 0.101,
#             -16 => -2.0E19, 16 => 2.0E19))
        
    catch e
        #rethrow(e)
        plot()
    end
end

In [ ]:
using Printf
a = @animate for (k,time0) in enumerate(dd.equilibrium.time[2:2:end])
    try
        FUSE.plot_plasma_overview(dd, Float64(time0); dd1, aggregate_hcd=true)#, min_power=1E4)

        IMAS.ylim(Dict{Int,Float64}(
            -2 => 0.0,
            -3 => 0.0, 3 => 2.0,
            -4 => 0.0, 4 => 4.0,
            -5 => 0.0, 5 => 4.0,
            6 => 1.1E20,

            -8 => -0.2, 8 => 2.0,
            -9 => -0.25, 9 => .5,
            -10 => -0.25, 10 => .5,
            -11 => -1E20, 11 => 1.0E20,

            -14 => 0.0, 14 => 0.101,
            -15 => 0.0, 15 => 0.101,
            -16 => -2.0E19, 16 => 2.0E19))

        # p=plot(dd1.equilibrium; time0, color=:black, label=" CAKE ")
        # plot!(ddF.equilibrium; time0, label=" FUSE - FRESCO",lw=2)
        # #plot!(ddE.equilibrium; time0, label=" FUSE - EGGO (ML) ", lw=2)
        # plot!(p[1], dd.wall, color=:black, legend_position = :outertop, legendfontsize=11)
        # #plot!(p[1], dd.pulse_schedule.position_control, nothing; time0)
        # plot!(p[1], ylim=(-1.5,1.5))
        # plot!(p[2], ylim=(0.0,0.1))
        # #annotate!(p[2], 0.5, 0.1, text("Pressure [MPa]", 14), title="")
        # #annotate!(p[3], 0.5, 2.2, text("Current [MA/m²]", 14), title="")
        # plot!(p[3], ylim=(0.0,2.3))

        # plot(dd.core_profiles; time0)

#         plot(dd.core_transport; time0)

        # plot(dd.core_sources; time0, aggregate_radiation=true, aggregate_hcd=true, only=5)

        savefig("D3D_$(shot)/D3D_$(shot)___$(@sprintf("%04d", k)).png") # magick -delay 2 -loop 0 D3D_168830___*.png -layers Optimize D3D_168830.gif
    catch e
        #rethrow(e)
        plot()
    end
end

In [ ]:
gif(a, "D3D_$(shot)/D3D_$(shot).gif", fps=12)